In [1]:
#r "nuget: Microsoft.ML"
#r "nuget: Microsoft.ML.LightGbm"
using System;
using System.IO;
using System.Linq;
using System.Net;
using Microsoft.ML;
using Microsoft.ML.Data;
using Microsoft.ML.Transforms;

Installed package Microsoft.ML.LightGbm version 1.4.0

Installed package Microsoft.ML version 1.4.0

In [2]:
if (!File.Exists("balance-scale.data"))
{
    using var client = new WebClient();
    client.DownloadFile("https://archive.ics.uci.edu/ml/machine-learning-databases/balance-scale/balance-scale.data", "balance-scale.data");
}

Console.WriteLine($"Data file has {File.ReadLines("balance-scale.data").Count():n0} lines");
File.ReadLines("balance-scale.data").Take(5)

Data file has 625 lines


index,value
0,"B,1,1,1,1"
1,"R,1,1,1,2"
2,"R,1,1,1,3"
3,"R,1,1,1,4"
4,"R,1,1,1,5"


> # Data Set Information:

> This data set was generated to model psychological experimental results. Each example is classified as having the balance scale tip to the right, tip to the left, or be balanced. The attributes are the left weight, the left distance, the right weight, and the right distance. The correct way to find the class is the greater of (left-distance * left-weight) and (right-distance * right-weight). If they are equal, it is balanced.

In [3]:
var context = new MLContext();

In [4]:
class BalanceScaleRow
{
    [ColumnName("Label")]
    [LoadColumn(0)]
    public string ClassName { get; set; }
    
    [LoadColumn(1)]
    public float LeftWeight { get; set; }
    
    [LoadColumn(2)]
    public float LeftDistance { get; set; }
    
    [LoadColumn(3)]
    public float RightWeight { get; set; }
    
    [LoadColumn(4)]
    public float RightDistance { get; set; }
}

class BalanceScaleData : BalanceScaleRow
{
    public float LeftFactor { get; set; }
    public float RightFactor { get; set; }
}

In [10]:
var allRows = context.Data.LoadFromTextFile<BalanceScaleRow>("balance-scale.data", hasHeader: false, separatorChar: ',');

var rowsEnum = context.Data.CreateEnumerable<BalanceScaleRow>(allRows, reuseRowObject: false);
var allData = rowsEnum.Select(d => new BalanceScaleData
{
    ClassName = d.ClassName,
    LeftWeight = d.LeftWeight,
    LeftDistance = d.LeftDistance,
    RightWeight = d.RightWeight,
    RightDistance = d.RightDistance,
    LeftFactor = d.LeftWeight * d.LeftDistance,
    RightFactor = d.RightWeight * d.RightDistance,
});

allData.Take(5)

index,LeftFactor,RightFactor,ClassName,LeftWeight,LeftDistance,RightWeight,RightDistance
0,1,1,B,1,1,1,1
1,1,2,R,1,1,1,2
2,1,3,R,1,1,1,3
3,1,4,R,1,1,1,4
4,1,5,R,1,1,1,5


In [11]:
var allDataView = context.Data.LoadFromEnumerable(allData);
allDataView = context.Data.ShuffleRows(allDataView);

In [12]:
var splitData = context.Data.TrainTestSplit(allDataView, testFraction: 0.2);
var (trainData, testData) = (splitData.TrainSet, splitData.TestSet);

In [16]:
var featureColumns = new[]
{
    nameof(BalanceScaleData.LeftWeight), nameof(BalanceScaleData.LeftDistance), nameof(BalanceScaleData.RightWeight), nameof(BalanceScaleData.RightDistance),
    nameof(BalanceScaleData.LeftFactor), nameof(BalanceScaleData.RightFactor)
};

In [17]:
trainData.GetColumn<string>("Label").Distinct()

index,value
0,R
1,B
2,L


In [18]:
var pipeline = 
    context.Transforms.Conversion.MapValueToKey("Label")
    .Append(context.Transforms.Conversion.MapKeyToValue("LabelValue", "Label"))
    .Append(context.Transforms.Concatenate("Features", featureColumns));

In [19]:
var transformer = pipeline.Fit(trainData);

In [21]:
class BalanceScaleDataTransformed
{
    [ColumnName("LabelValue")]
    public string Class { get; set; }

    [VectorType(84)]
    public float[] Features { get; set; }
}

var transformedData = transformer.Transform(trainData);
context.Data
    .CreateEnumerable<BalanceScaleDataTransformed>(transformedData, reuseRowObject: false)
    .Take(3)

index,Class,Features
0,R,"[ 1, 1, 2, 3, 1, 6 ]"
1,B,"[ 1, 3, 1, 3, 3, 3 ]"
2,R,"[ 5, 3, 5, 4, 15, 20 ]"


In [22]:
var estimator = context.MulticlassClassification.Trainers.LightGbm(featureColumnName: "Features");

In [23]:
var transformedTrainData = transformer.Transform(trainData);
var cvResults = context.MulticlassClassification.CrossValidate(transformedTrainData, estimator, numberOfFolds: 3);
var cvResult = cvResults
    .OrderByDescending(x => x.Metrics.MacroAccuracy)
    .First();

In [24]:
cvResult.Metrics.ConfusionMatrix.GetFormattedConfusionTable()


Confusion table
          ||========================
PREDICTED ||     R |     B |     L | Recall
TRUTH     ||========================
        R ||    69 |     5 |     0 | 0.9324
        B ||     4 |     6 |     4 | 0.4286
        L ||     0 |     4 |    72 | 0.9474
          ||========================
Precision ||0.9452 |0.4000 |0.9474 |


In [25]:
cvResult.Metrics

LogLoss,LogLossReduction,MacroAccuracy,MicroAccuracy,TopKAccuracy,TopKPredictionCount,PerClassLogLoss,ConfusionMatrix
0.2972664631089637,0.6788314292297085,0.7694574273521643,0.8963414634146342,0,0,"[ 0.19312904676728335, 1.8183831710018639, 0.11845771177717082 ]","{ Microsoft.ML.Data.ConfusionMatrix: PerClassPrecision: [ 0.9452054794520548, 0.4, 0.9473684210526315 ], PerClassRecall: [ 0.9324324324324325, 0.42857142857142855, 0.9473684210526315 ], Counts: [ [ 69, 5, 0 ], [ 4, 6, 4 ], [ 0, 4, 72 ] ], NumberOfClasses: 3 }"


In [26]:
var transformedTestData = transformer.Transform(testData);
var predictions = cvResult.Model.Transform(transformedTestData);
var metrics = context.MulticlassClassification.Evaluate(predictions);
metrics.ConfusionMatrix.GetFormattedConfusionTable()


Confusion table
          ||========================
PREDICTED ||     R |     B |     L | Recall
TRUTH     ||========================
        R ||    55 |     4 |     0 | 0.9322
        B ||     2 |     1 |     3 | 0.1667
        L ||     0 |     2 |    52 | 0.9630
          ||========================
Precision ||0.9649 |0.1429 |0.9455 |


In [27]:
metrics

LogLoss,LogLossReduction,MacroAccuracy,MicroAccuracy,TopKAccuracy,TopKPredictionCount,PerClassLogLoss,ConfusionMatrix
0.24925543418599327,0.7091600530747111,0.6872776731533793,0.907563025210084,0,0,"[ 0.14410143518141205, 2.434462829728304, 0.12134509285296427 ]","{ Microsoft.ML.Data.ConfusionMatrix: PerClassPrecision: [ 0.9649122807017544, 0.14285714285714285, 0.9454545454545454 ], PerClassRecall: [ 0.9322033898305084, 0.16666666666666666, 0.9629629629629629 ], Counts: [ [ 55, 4, 0 ], [ 2, 1, 3 ], [ 0, 2, 52 ] ], NumberOfClasses: 3 }"


In [32]:
class MulticlassClassificationPrediction
{
    public string LabelValue { get; set; }

    public float[] Score { get; set; }

    public string PredictedLabelValue { get; set; }
}

var sampleData = context.Data.ShuffleRows(testData);
var transformedSampleData = transformer.Transform(sampleData);

var samplePredictions = cvResult.Model.Transform(transformedSampleData);
var mapValues = context.Transforms.Conversion
    .MapKeyToValue("PredictedLabelValue", "PredictedLabel")
    .Append(context.Transforms.Conversion.MapKeyToValue("LabelValue", "Label"))
    .Fit(samplePredictions);
samplePredictions = mapValues.Transform(samplePredictions);
var samplePredictionItems = context.Data.CreateEnumerable<MulticlassClassificationPrediction>(samplePredictions, reuseRowObject: false);

samplePredictionItems.Take(5)

index,LabelValue,Score,PredictedLabelValue
0,R,"[ 0.9916059, 0.00834077, 5.3353237E-05 ]",R
1,R,"[ 0.998851, 0.0010052732, 0.00014369821 ]",R
2,L,"[ 0.00012103434, 0.00055790815, 0.99932104 ]",L
3,R,"[ 0.9977286, 0.0022037618, 6.766315E-05 ]",R
4,B,"[ 0.33228013, 0.23664351, 0.43107638 ]",L
